In [19]:
from pysat.solvers import Glucose4
from pysat.card import *
import numpy as np

import sys
data = []
#data = sys.stdin.readlines()
with open("statement.jfp") as f:
    data = f.readlines()



class JobFlowProblem:
    
    def __init__(self, m, j):
        self.machines = m
        self.jobs = j
        self.tasks = [0 for i in range(m*j)]
        self.max_timestep = 20
        
    def __str__(self):
        sm = ""
        for j in range(self.jobs):
            for i in range(self.machines):
                sm += " " + str(self.tasks[self.t_index(i,j)])
            sm += "\n"
        return sm
    
    def set_task(self, m, j, d):
        self.tasks[self.t_index(m,j)] = d
        return
    
    def t_index(self, m, j):
        return m * self.jobs + j

    def var_id(self, m, job, time):
        return m * self.jobs * self.max_timestep + job * self.max_timestep + time + 1
    
    def id_var(self, var_id):
        var_id = var_id - 1 
        t = var_id % self.max_timestep
        var_id -= t
        j = ( var_id % (self.jobs * self.max_timestep) ) // self.max_timestep
        var_id -= j * self.max_timestep
        m = var_id // (self.jobs*self.max_timestep)
        return m,j,t
        
    
    def solve(self):
        g = Glucose4()
        #DUAS TAREFAS NÃO SE PODEM SOBREPOR NA MESMA MAQUINA
        for j in range(0,self.jobs):
            for m in range(0, self.machines):
                for t in range(0, self.max_timestep):
                    if t+self.tasks[self.t_index(m,j)] < self.max_timestep:
                        for d in range(t, t+self.tasks[self.t_index(m,j)]):
                            for other_j in range(0, self.jobs):
                                if other_j != j:
                                    g.add_clause([-self.var_id(m, j, t), -self.var_id(m, other_j, d)])
                                
        #TODAS AS TAREFAS TÊM DE SER EXECUTADAS
        for m in range(0, self.machines):
            for j in range(0,self.jobs):
                lst = []
                for t in range(0, self.max_timestep):
                    if t+self.tasks[self.t_index(m,j)] < self.max_timestep:
                        lst += [self.var_id(m,j,t)]
                
                cnf = CardEnc.equals(lits=lst, encoding=EncType.pairwise)
                g.append_formula(cnf.clauses)
        
                
        #TAREFAS SÃO SEQUENCIAIS
        for m in range(0, self.machines):
            for j in range(0,self.jobs):
                for t in range(0, self.max_timestep):
                    if t+self.tasks[self.t_index(m,j)] < self.max_timestep:
                        for other_t in range(0, t+self.tasks[self.t_index(m,j)]):
                            for other_m in range(m+1, self.machines):
                                if other_m != m:
                                    g.add_clause([-self.var_id(m, j, t), -self.var_id(other_m, j, other_t)])
                
        model = None
        sat = True
        t = self.max_timestep
        while sat and t >= 0:
            asmpt = []
            for m in range(self.machines):
                for j in range(self.jobs):
                    for t1 in range(t, self.max_timestep):
                        asmpt.append(-self.var_id(m,j,t1))
            
            sat = g.solve(asmpt)
            if sat:
                model = g.get_model()
                t -= 1
            
        print(t+1)
        print(self.jobs, self.machines)
        
        result = self.parse_model(model)
        
        for j in range(self.jobs):
            print((result[:, j] >= 0).sum(), end=' ')
            for m in range(self.machines):
                print(str(m) + ':' + str(result[m][j]), end=' ')
            print()
        
        
    def parse_model(self, model):
        result = np.full((self.machines, self.jobs), -1, dtype=int)
        
        for var in model:
            m, j, t = self.id_var(abs(var))
            if var > 0:
                result[m][j] = t
        
        return result
    
    
    def print_model(self, model):
        if not model:
            print("No solution!")
            return
        
        out = np.zeros((self.machines, self.max_timestep), dtype=int)
        
        for var in model:
            sign = var > 0
            
            m, j, t = self.id_var(abs(var))
            if sign:
                out[m][t] = j+1
                for t1 in range(t, t+self.tasks[self.t_index(m,j)]):
                    out[m][t1] = j+1
            
        for m in range(self.machines):
            print("m" + '{0:01d}'.format(m+1) + " ", end='')
            for t in range(self.max_timestep):
                if out[m][t] != 0:
                    print('j' + str(out[m][t]), end=' ')
                else:
                    print("   ", end='')
            print()
        
        print("t  ", end='')
        for t in range(self.max_timestep):
            print('{0:02d} '.format(t), end = '')
    

n, m = map(int, data[0].split())

problem = JobFlowProblem(m, n)

for job in range(0, n):
    parts = data[job+1].split()
    k = int(parts[0])
    
    for i in range(0, k):
        machine, duration = map(int,parts[i+1].split(":"))
        problem.set_task(machine-1, job, duration)
    

problem.solve()

8
3 2
2 0:0 1:2 
2 0:4 1:7 
2 0:2 1:4 
